## 事件流处理器

In [1]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


### 推送事件流

In [2]:
async for x in r(generator="async", verbose=False):
    print("SSE >>", x)

hi
SSE >> {'event': 'text', 'id': '20241120-33201-0010-6103', 'data': {'content': 'hi\n', 'content_id': '20241120-33201-0011-6353', 'thread_id': None, 'calling_id': '20241120-33201-0001-4496', 'agent_name': 'MyRun.4598540528', 'model_name': None, 'created_at': '2024-11-20T00:20:01.881969'}}
illufly!
SSE >> {'event': 'text', 'id': '20241120-33201-0014-2500', 'data': {'content': 'illufly!\n', 'content_id': '20241120-33201-0015-9489', 'thread_id': None, 'calling_id': '20241120-33201-0001-4496', 'agent_name': 'MyRun.4598540528', 'model_name': None, 'created_at': '2024-11-20T00:20:01.882403'}}


In [3]:
async for x in r(generator="async", verbose=False):
    print("SSE >>", x)

hi
SSE >> {'event': 'text', 'id': '20241120-33204-0020-0007', 'data': {'content': 'hi\n', 'content_id': '20241120-33204-0021-9862', 'thread_id': None, 'calling_id': '20241120-33204-0002-4372', 'agent_name': 'MyRun.4598540528', 'model_name': None, 'created_at': '2024-11-20T00:20:04.020425'}}
illufly!
SSE >> {'event': 'text', 'id': '20241120-33204-0024-1159', 'data': {'content': 'illufly!\n', 'content_id': '20241120-33204-0025-3211', 'thread_id': None, 'calling_id': '20241120-33204-0002-4372', 'agent_name': 'MyRun.4598540528', 'model_name': None, 'created_at': '2024-11-20T00:20:04.021055'}}


In [4]:
r.events_history.store

{'20241120-33201-0000-3043': {'threads': set(),
  'callings': {'20241120-33201-0000-6428': {'agent_name': 'MyRun.4598540528',
    'input': '',
    'output': '',
    'segments': {'20241120-33201-0003-5620': 'hi\n',
     '20241120-33201-0005-9284': 'illufly!\n'},
    'other_events': []},
   '20241120-33201-0001-4496': {'agent_name': 'MyRun.4598540528',
    'input': '',
    'output': '',
    'segments': {'20241120-33201-0009-7866': 'hi\n',
     '20241120-33201-0013-6729': 'illufly!\n'},
    'other_events': []},
   '20241120-33204-0002-4372': {'agent_name': 'MyRun.4598540528',
    'input': '',
    'output': '',
    'segments': {'20241120-33204-0019-1807': 'hi\n',
     '20241120-33204-0023-0522': 'illufly!\n'},
    'other_events': []}}}}

In [9]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [10]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun(handlers=[handle])
r("hi")

{"id": "c3cd3fc0a4ba11ef9c7aacde48001122", "content_id": "c3cd4042a4ba11ef9c7aacde48001122", "block_type": "runnable", "content": "MyRun.4592974432", "created_at": "2024-11-17T16:05:52.765559", "calling_info": {}, "runnable_info": {"name": "MyRun.4592974432", "class_name": "MyRun", "calling_id": "c3cd3d54-a4ba-11ef-9c7a-acde48001122"}}
{"id": "c3cd5aaaa4ba11ef9c7aacde48001122", "content_id": "c3cd5bb8a4ba11ef9c7aacde48001122", "block_type": "chunk", "content": "hi, ", "created_at": "2024-11-17T16:05:52.766262", "calling_info": {}, "runnable_info": {"calling_id": "c3cd3d54-a4ba-11ef-9c7a-acde48001122"}}
{"id": "c3cd5fb4a4ba11ef9c7aacde48001122", "content_id": "c3cd6022a4ba11ef9c7aacde48001122", "block_type": "chunk", "content": "illufly", "created_at": "2024-11-17T16:05:52.766372", "calling_info": {}, "runnable_info": {"calling_id": "c3cd3d54-a4ba-11ef-9c7a-acde48001122"}}
{"id": "c3cd628ea4ba11ef9c7aacde48001122", "content_id": "c3cd62e8a4ba11ef9c7aacde48001122", "block_type": "chunk",

### ChatAgent 事件流

In [ ]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

def handle(block, **kwargs):
    print("[", block.block_type, "]", block.text)

r = FakeLLM(handlers=[handle], sleep=0.5)
r("hi")

In [ ]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM
from illufly.io import log, alog

r = FakeLLM(handlers=[alog], sleep=0.1)
await r("hi")

### 推送 ChatAgent 事件流

In [10]:
from illufly.chat import FakeLLM

llm = FakeLLM(name="if1", handlers=[])
llm.create_new_thread()

async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'event': 'info', 'id': 'aec6e7b8a68e11efac23acde48001122', 'data': {'content': '记住 10 轮对话', 'content_id': 'aec6e808a68e11efac23acde48001122', 'thread_id': '20241119-31721-0000-0948', 'calling_id': '20241119-31721-0000-0913', 'agent_name': 'if1', 'model_name': None, 'created_at': '2024-11-19T23:55:21.961274'}}
{'event': 'info', 'id': 'aec7383aa68e11efac23acde48001122', 'data': {'content': 'I am FakeLLM', 'content_id': 'aec7115ca68e11efac23acde48001122', 'thread_id': '20241119-31721-0000-0948', 'calling_id': '20241119-31721-0000-0913', 'agent_name': 'if1', 'model_name': None, 'created_at': '2024-11-19T23:55:21.963365'}}
{'event': 'chunk', 'id': 'aed72dd0a68e11efac23acde48001122', 'data': {'content': 'R', 'content_id': 'aec7115ca68e11efac23acde48001122', 'thread_id': '20241119-31721-0000-0948', 'calling_id': '20241119-31721-0000-0913', 'agent_name': 'if1', 'model_name': None, 'created_at': '2024-11-19T23:55:22.067992'}}
{'event': 'chunk', 'id': 'aee6905ea68e11efac23acde48001122', 'data':

In [9]:
async for b in llm("给我写一篇200字小作文", generator="async", sleep=0.1):
    print(b)

{'event': 'info', 'id': 'aa267d36a68e11efac23acde48001122', 'data': {'content': '记住 10 轮对话', 'content_id': 'aa267d9aa68e11efac23acde48001122', 'thread_id': '20241119-31683-0000-5375', 'calling_id': '20241119-31714-0000-1194', 'agent_name': 'if1', 'model_name': None, 'created_at': '2024-11-19T23:55:14.199087'}}
{'event': 'info', 'id': 'aa26dd12a68e11efac23acde48001122', 'data': {'content': 'I am FakeLLM', 'content_id': 'aa26a86aa68e11efac23acde48001122', 'thread_id': '20241119-31683-0000-5375', 'calling_id': '20241119-31714-0000-1194', 'agent_name': 'if1', 'model_name': None, 'created_at': '2024-11-19T23:55:14.201569'}}
{'event': 'chunk', 'id': 'aa363302a68e11efac23acde48001122', 'data': {'content': 'R', 'content_id': 'aa26a86aa68e11efac23acde48001122', 'thread_id': '20241119-31683-0000-5375', 'calling_id': '20241119-31714-0000-1194', 'agent_name': 'if1', 'model_name': None, 'created_at': '2024-11-19T23:55:14.302114'}}
{'event': 'chunk', 'id': 'aa46188aa68e11efac23acde48001122', 'data':

## 嵌套 BaseAgent

In [ ]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

store = {}
flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking])
)

for b in flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False, handlers=[], generator="sync"):
    print(b)

## Events History

In [14]:
from illufly.core.history.events import BaseEventsHistory

In [20]:
h = BaseEventsHistory()

In [23]:
h.store

{'20241117-28722-2213-0000': {'threads': set(), 'callings': {}}}

In [24]:
h.last_events_history_id

'20241117-28722-2213-0000'

In [25]:
24*60*60

86400